In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pandas._libs.lib import is_integer
import re

In [2]:
tone_file = "final_tone_analysis_result.csv" # 문서 어조 점수 파일
macro_file = "macro_data.csv" # 경제 지표 파일

In [3]:
# 1. 데이터로드
print(">>> 1. 문서별 어조 지수 파일 로드 중...")
df_tone = pd.read_csv(tone_file)

def extract_date_from_id(doc_id):
    # doc_id 안에 있는 연속된 숫자 8개(YYYYMMDD) 찾기
    match = re.search(r"(\d{8})", str(doc_id))
    if match:
        return match.group(1)
    return None

# 날짜 추출
df_tone['DATE'] = df_tone['doc_id'].apply(extract_date_from_id)

# datetime 형식으로 변환 (오류 발생 시 NaT 처리)
df_tone['DATE'] = pd.to_datetime(df_tone['DATE'], format='%Y%m%d', errors='coerce')

# 날짜가 없는 행 제거 (안전장치)
# df_tone = df_tone.dropna(subset=['DATE'])
print(f"   - 날짜 변환 완료. 데이터 기간: {df_tone['DATE'].min().date()} ~ {df_tone['DATE'].max().date()}")


# 2. FOMC 의사록만 필터링
print("\n>>> 2. FOMC 의사록 필터링 중...")

# doc_id가 'fomc'로 시작하는지 확인
def is_minutes(doc_id):
    return str(doc_id).strip().startswith('FOMC')

df_target = df_tone[df_tone['doc_id'].apply(is_minutes)].copy()

# 날짜순 정렬 (시계열 분석을 위해 필수)
df_target = df_target.sort_values('DATE')

print(f"   - 전체 문서 개수: {len(df_tone)}개")
print(f"   - 'FOMC' 의사록 개수: {len(df_target)}개")
print("\n[데이터 미리보기]")
print(df_target[['DATE', 'doc_id', 'Market_Tone']].head())

>>> 1. 문서별 어조 지수 파일 로드 중...
   - 날짜 변환 완료. 데이터 기간: 2013-09-18 ~ 2025-10-09

>>> 2. FOMC 의사록 필터링 중...
   - 전체 문서 개수: 29949개
   - 'FOMC' 의사록 개수: 97개

[데이터 미리보기]
        DATE         doc_id  Market_Tone
0 2013-09-18  FOMC_20130918     0.392405
1 2013-10-30  FOMC_20131030     0.466667
2 2013-12-18  FOMC_20131218     0.328358
3 2014-01-29  FOMC_20140129     0.596730
4 2014-03-19  FOMC_20140319     0.455224


In [4]:
# (3) 월별 평균 어조 지수 산출
df_target['YearMonth'] = df_target['DATE'].dt.to_period('M').dt.to_timestamp()
monthly_tone = df_target.groupby('YearMonth')[['Market_Tone', 'Dictionary_Tone']].mean().reset_index()

In [5]:
# (4) 경제 데이터 로드
df_macro = pd.read_csv('macro_data.csv')
df_macro['DATE'] = pd.to_datetime(df_macro['DATE'])
df_macro['YearMonth'] = df_macro['DATE'].dt.to_period('M').dt.to_timestamp()

In [6]:
# 2. 데이터 병합 및 변수 생성 - 테일러 준칙용

# 날짜 기준으로 병합
df = pd.merge(df_macro, monthly_tone, on='YearMonth', how='inner').sort_values('YearMonth')

# 변수 생성 (전월 대비 변화량)
df['Delta_Base_Rate'] = df['Base_Rate'].diff()                # 금리 변화 (Y)
df['Lag_Delta_Base'] = df['Delta_Base_Rate'].shift(1)         # 지난달 금리 변화
df['Delta_Inflation'] = df['CPI'].diff()                      # 물가 변화
df['Delta_Output'] = df['IP'].pct_change() * 100              # 경기 변화 (IP:산업생산지수)

df = df.dropna() # 결측치 제거

In [7]:
# 3. 회귀분석 수행 및 결과 출력
target_tones = ['Market_Tone', 'Dictionary_Tone']

print("="*50)
print("             [최종 회귀분석 결과]             ")
print("="*50)

for tone_col in target_tones:
    if tone_col not in df.columns: continue

    # X: 설명변수 (상수항 포함)
    X = df[['Lag_Delta_Base', 'Delta_Inflation', 'Delta_Output', tone_col]]
    X = sm.add_constant(X)
    y = df['Delta_Base_Rate']

    # 모델 학습
    model = sm.OLS(y, X).fit()

    # 결과 요약 출력
    print(f"\n 분석 대상: {tone_col}")
    print(f"   - R-squared (설명력) : {model.rsquared:.4f}")
    print(f"   - Coef (계수)        : {model.params[tone_col]:.4f}")
    print(f"   - P-value (유의확률) : {model.pvalues[tone_col]:.4f}")

    if model.pvalues[tone_col] < 0.05:
        print("   ✅ 통계적으로 유의미함 (가설 채택)")
    else:
        print("   ")

print("\n" + "="*50)

             [최종 회귀분석 결과]             

 분석 대상: Market_Tone
   - R-squared (설명력) : 0.5007
   - Coef (계수)        : -0.0423
   - P-value (유의확률) : 0.8558
   

 분석 대상: Dictionary_Tone
   - R-squared (설명력) : 0.5209
   - Coef (계수)        : 0.5486
   - P-value (유의확률) : 0.0536
   

